In [135]:
from DbConnector import DbConnector
#from part2 import Database
from tabulate import tabulate

# Set up the program
from dotenv import load_dotenv
import os
import numpy as np
from datetime import datetime, timedelta
from haversine import haversine, Unit

load_dotenv()

db = DbConnector()

Using user:  common
Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assignment2',)
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

In [116]:
result: list = []

# For each table, retrieve the amount of rows it has
for table_name in ["User", "Activity", "TrackPoint"]:
    query = "SELECT COUNT(*) FROM %s"
    db.cursor.execute(query % table_name)
    result.append(db.cursor.fetchone())

print("Number of Users, Activities and TrackPoints after the dataset has been inserted into the database")
print(tabulate([["Users", "Activities", "TrackPoints"], [result[0][0], result[1][0], result[2][0]]], headers="firstrow"))

Number of Users, Activities and TrackPoints after the dataset has been inserted into the database
  Users    Activities    TrackPoints
-------  ------------  -------------
    182          7877        5355109


### Task 2
Find the average, maximum and minimum number of trackpoints per user

##### Average

In [117]:
# Get the average number of trackpoints per user

# The first (#1) SELECT is to count the amount of trackpoints for each activity of each user that has at least one trackpoint:
# SELECT Activity.user_id , COUNT(t.id) AS trackpoints FROM TrackPoint t 
# JOIN Activity ON t.activity_id = Activity.id JOIN User ON Activity.user_id = User.id 
# GROUP BY Activity.id

# The second (#2) SELECT is to calculate the average for each user
# SELECT user_id, AVG(trackpoints) average 
# FROM (#1) AS Trackpoints GROUP BY user_id)

# THe third SELECT is to set users with no trackpoints the value 0
# SELECT User.id, COALESCE(average, 0) AS average 
# FROM User LEFT JOIN (#2) a ON a.user_id = User.id

query = """
SELECT User.id, COALESCE(average, 0) AS average 
    FROM User LEFT JOIN (SELECT user_id, AVG(trackpoints) average 
    FROM (SELECT Activity.user_id , COUNT(t.id) AS trackpoints 
    FROM TrackPoint t JOIN Activity ON t.activity_id = Activity.id 
    JOIN User ON Activity.user_id = User.id GROUP BY Activity.id) 
AS Trackpoints GROUP BY user_id) a ON a.user_id = User.id"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Average trackpoints per user")
print(tabulate(rows, headers=db.cursor.column_names))


Average trackpoints per user
  id    average
----  ---------
 000   670.871
 001   824.965
 002   924.801
 003   807.387
 004   761.858
 005   587.548
 006   801.125
 007   758.275
 008  1376
 009  1218.32
 010  1107.5
 011   451.756
 012   629.783
 013  1388.3
 014   905.936
 015   806.583
 016   690.222
 017   868.245
 018   540.795
 019   372.291
 020   704.638
 021   475
 022   900.695
 023   979.312
 024   860.408
 025   606.295
 026  1034.71
 027  2037
 028  1195.36
 029  1713.17
 030   871.352
 031   489.5
 032  1190.43
 033   735.25
 034   558.678
 035  1449.71
 036   803.75
 037   890.977
 038  1071.51
 039   756.51
 040   953.55
 041   797.416
 042   555.946
 043   795.094
 044   628.492
 045   752.125
 046   703.807
 047    65.8333
 048  1118
 049     0
 050   760.833
 051   231.3
 052   177
 053     0
 054  1093
 055   132.526
 056    39.7333
 057   127
 058    38.5
 059     0
 060     8
 061   140
 062   439.054
 063   709.857
 064   695
 065  1114.94
 066   889.375
 067  

##### Maximum

In [118]:
# Get the maximum number of trackpoints per user

# The first (#1) SELECT is to count the amount of trackpoints for each activity of each user that has at least one trackpoint:
# SELECT Activity.user_id , COUNT(t.id) AS trackpoints FROM TrackPoint t 
# JOIN Activity ON t.activity_id = Activity.id JOIN User ON Activity.user_id = User.id 
# GROUP BY Activity.id

# The second (#2) SELECT is to calculate the maximum trackpoints for each user
# SELECT user_id, MAX(trackpoints) average 
# FROM (#1) AS Trackpoints GROUP BY user_id)

# THe third SELECT is to set users with no trackpoints the value 0
# SELECT User.id, COALESCE(maximum, 0) AS maximum 
# FROM User LEFT JOIN (#2) a ON a.user_id = User.id

query = """SELECT User.id, COALESCE(maximum, 0) AS maximum 
                FROM User LEFT JOIN (SELECT user_id, MAX(trackpoints) AS maximum 
                FROM (SELECT Activity.user_id , COUNT(t.id) AS trackpoints FROM TrackPoint t 
                INNER JOIN Activity ON t.activity_id = Activity.id GROUP BY Activity.id) 
                AS Trackpoints GROUP BY user_id) a ON a.user_id = User.id"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Maximum trackpoints per user")
print(tabulate(rows, headers=db.cursor.column_names))

Maximum trackpoints per user
  id    maximum
----  ---------
 000       2359
 001       2472
 002       2438
 003       2485
 004       2482
 005       2058
 006       2478
 007       2228
 008       2499
 009       2396
 010       1964
 011       2306
 012       2277
 013       2486
 014       2499
 015       2411
 016       2360
 017       2471
 018       2245
 019       2276
 020       2201
 021        475
 022       2421
 023       2215
 024       2377
 025       2464
 026       2396
 027       2480
 028       2477
 029       2440
 030       2454
 031       1254
 032       2358
 033       1205
 034       2448
 035       2493
 036       2299
 037       2421
 038       2500
 039       2464
 040       2434
 041       2491
 042       2350
 043       2005
 044       2206
 045       1932
 046       2457
 047        227
 048       1118
 049          0
 050       2484
 051        951
 052        177
 053          0
 054       1527
 055        670
 056        165
 057        510
 058       

##### Minimum

In [119]:
# Get the minimum number of trackpoints per user

# The first (#1) SELECT is to count the amount of trackpoints for each activity of each user that has at least one trackpoint:
# SELECT Activity.user_id , COUNT(t.id) AS trackpoints FROM TrackPoint t 
# JOIN Activity ON t.activity_id = Activity.id JOIN User ON Activity.user_id = User.id 
# GROUP BY Activity.id

# The second (#2) SELECT is to calculate the minimum trackpoints for each user
# SELECT user_id, MIN(trackpoints) average 
# FROM (#1) AS Trackpoints GROUP BY user_id)

# THe third SELECT is to set users with no trackpoints the value 0
# SELECT User.id, COALESCE(minimum, 0) AS minimum 
# FROM User LEFT JOIN (#2) a ON a.user_id = User.id

query = """
        SELECT User.id, COALESCE(minimum, 0) AS minimum FROM User LEFT JOIN (SELECT user_id, MIN(trackpoints) AS minimum
            FROM (SELECT Activity.user_id , COUNT(t.id) AS trackpoints 
            FROM TrackPoint t
            INNER JOIN Activity ON t.activity_id = Activity.id 
            GROUP BY Activity.id) 
            AS Trackpoints 
            GROUP BY user_id) a ON a.user_id = User.id
            """
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Minimum trackpoints per user")
print(tabulate(rows, headers=db.cursor.column_names))

Minimum trackpoints per user
  id    minimum
----  ---------
 000          5
 001         33
 002          4
 003          3
 004          4
 005          5
 006         14
 007          6
 008        165
 009        134
 010        663
 011          3
 012         64
 013         13
 014          3
 015         22
 016          7
 017          4
 018          3
 019         11
 020         22
 021        475
 022         15
 023         31
 024          4
 025          3
 026          9
 027       1760
 028         32
 029          4
 030          9
 031         23
 032         62
 033         39
 034          4
 035        200
 036         13
 037         14
 038          8
 039          7
 040         39
 041          3
 042          4
 043         26
 044          3
 045         71
 046        175
 047          4
 048       1118
 049          0
 050          3
 051          5
 052        177
 053          0
 054        659
 055          5
 056         16
 057          9
 058       

### Task 3

Find the top 15 users with the highest number of activities.

In [120]:
# Used RANK() to get the top ordering column from 1-15 based on the COUNT(*)
query = """
    SELECT RANK() OVER (
        ORDER BY COUNT(*) DESC
    ) Top, user_id, COUNT(*) as num_of_activities FROM Activity GROUP BY user_id ORDER BY COUNT(*) DESC LIMIT 15
    """
db.cursor.execute(query)
rows = db.cursor.fetchall()
columns = db.cursor.column_names
print("Top 15 users with the highest number of activities")
print(tabulate(rows, headers=columns))

Top 15 users with the highest number of activities
  Top    user_id    num_of_activities
-----  ---------  -------------------
    1        025                  715
    2        128                  519
    3        062                  406
    4        041                  399
    5        004                  346
    6        140                  345
    7        017                  265
    8        003                  261
    9        014                  236
   10        030                  210
   11        011                  201
   12        039                  198
   13        034                  180
   14        000                  155
   15        002                  146


### Task 4

Find all users who have taken a bus.

In [121]:
# Get the users who have taken the bus
query = """
SELECT DISTINCT user_id
FROM Activity
WHERE transportation_mode = 'bus'
"""
db.cursor.execute(query)
users = db.cursor.fetchall()

table = [["User id"]]
table.extend([user for user in users])
print("All users who have taken a bus")
print(tabulate(table, headers="firstrow"))

All users who have taken a bus
  User id
---------
      010
      052
      062
      073
      081
      084
      085
      091
      092
      112
      125
      128
      175


### Task 5

List the top 10 users by their amount of different transportation modes.

In [122]:
# Used RANK() to get the top ordering column from 1-15 based on the COUNT
query = """SELECT RANK() OVER (
        ORDER BY COUNT(DISTINCT(transportation_mode)) DESC
        ) AS Top, user_id, COUNT(DISTINCT(transportation_mode)) as DifferentTransportation 
                    FROM Activity GROUP BY user_id ORDER BY DifferentTransportation DESC LIMIT 10;"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Top 10 users by their amount of different transportation modes")
print(tabulate(rows, headers=db.cursor.column_names))

Top 10 users by their amount of different transportation modes
  Top    user_id    DifferentTransportation
-----  ---------  -------------------------
    1        128                          9
    2        062                          7
    3        085                          4
    4        058                          3
    4        078                          3
    4        081                          3
    4        084                          3
    4        112                          3
    4        163                          3
   10        010                          2


### Task 6

Find activities that are registered multiple times. You should find the query even
if it gives zero result.

Assumption: Registered multiple times, meaning duplicates with the same user_id, transportation_mode, start_date_time, end_date_time.

In [123]:
# Uses WHERE EXISTS to find another activity which is a duplicate
# add the AND constraint a.id != b.id such that it does not consider the same activity
query = """
    SELECT a.id FROM Activity AS a WHERE EXISTS 
        (SELECT b.id FROM Activity AS b WHERE a.user_id = b.user_id 
        AND a.transportation_mode = b.transportation_mode AND 
        a.start_date_time = b.start_date_time AND a.end_date_time = b.end_date_time 
        AND a.id != b.id)
    """
db.cursor.execute(query)
rows = db.cursor.fetchall()
columns = db.cursor.column_names
print("Activities that are registered multiple times")
print(tabulate(rows, headers=columns))

Activities that are registered multiple times
id
----


### Task 7

a) Find the number of users that have started an activity in one day and ended the activity the next day.

In [124]:
# Uses DATEDIFF to find the difference in day if larger than 0, then it means it ended on the next day
query = """
SELECT COUNT(DISTINCT user_id) 
FROM Activity 
WHERE DATEDIFF(end_date_time, start_date_time) > 0;
"""
db.cursor.execute(query)
num_users = db.cursor.fetchone()

print("Number of users with activity that ends the next day")
print(tabulate([["Number of users"], [num_users[0]]], headers="firstrow"))

Number of users with activity that ends the next day
  Number of users
-----------------
               66


b) List the transportation mode, user id and duration for these activities.

Assumption: As the question explicitly mentions transportation mode we do not count the rows in the Activity table with transportation mode equals NULL as relevant to this question. Thus, there are less rows in this sub task. However, if it is the case to include these NULL rows then we only need to remove the 'AND transportation_mode IS NOT NULL' part.

In [125]:
# Using the same logic as above, but now retrieving more information
query = """
SELECT 
    user_id, 
    transportation_mode, 
    SEC_TO_TIME(TIMESTAMPDIFF(SECOND, start_date_time, end_date_time))
FROM Activity 
WHERE DATEDIFF(end_date_time, start_date_time) > 0
AND transportation_mode IS NOT NULL;
"""
db.cursor.execute(query)
users_info = db.cursor.fetchall()

table = [["User", "Transportation Mode", "Duration"]]
content = [[user[0], user[1], user[2]] for user in users_info]
table.extend(content)

print(tabulate(table, headers="firstrow"))

  User  Transportation Mode    Duration
------  ---------------------  ----------
   020  bike                   10:10:01
   021  walk                   3:57:13
   058  car                    0:36:12
   062  walk                   1:23:44
   085  bus                    0:58:25
   115  car                    1:28:04
   115  car                    1:02:50
   115  car                    1:00:56
   115  car                    1:05:10
   115  car                    1:23:14
   115  car                    0:54:09
   115  car                    1:08:43
   115  car                    1:09:36
   115  car                    1:03:47
   115  car                    1:01:42
   115  car                    0:55:50
   115  car                    1:24:08
   115  car                    1:17:13
   115  car                    1:01:33
   115  car                    1:03:29
   115  car                    0:58:19
   115  car                    7:46:29
   115  car                    1:05:39
   115  car         

### Task 8

Find the number of users which have been close to each other in time and space. Close is defined as the same space (50 meters) and for the same half minute (30 seconds)

In [126]:
# Sorting the trackpoints by date_time to make it fast to find the datepoints within 30 seconds
query = """SELECT a.user_id, date_time, lat,lon FROM TrackPoint t
INNER JOIN Activity a
ON a.id = t.activity_id
ORDER BY date_time ASC"""

# Looping through all the squares
users = []
db.cursor.execute(query)
rows = db.cursor.fetchall()


for i in range(len(rows)-1):
    for j in range(i,len(rows)):
        # Skip the rest of the loop if the time difference is more than 30 seconds and they are sorted
        if rows[j][1] - rows[i][1] > timedelta(seconds=30):
            break
        # Dont do anything if the same user
        if rows[i][0] == rows[j][0]:
            continue
        # Checking if the trackpoints are within 30 seconds of each other
        if rows[j][1] - rows[i][1] <= timedelta(seconds=30):

            # Checking if the trackpoints are within 50 meters of each other
            distance = haversine((rows[i][2], rows[i][3]), (rows[j][2], rows[j][3]), unit=Unit.METERS)

            if distance <= 50:
                # Adding the users to the list
                users.append(rows[i][0])
                users.append(rows[j][0])


# Removes duplicates
users = np.unique(users)

print(f"{len(users)} have been close to each other in time and space")

81 have been close to each other in time and space


### Task 9

Find the top 15 users who have gained the most altitude meters.

In [127]:
# 1. Retrieve all the trackpoints which is valid
# 2. Calculate the difference between the current trackpoint's altitude and the previous one
# 3. Calculate the sum of difference if the difference is postive based on the tip given in task for each activity
# 4. Sum each user's altitude positive differences in feet with 0.304 to convert it to meters

# First select (#1) retrieves all the trackpoints that are valid. (valid altitude value):
# SELECT activity_id, altitude, date_time FROM TrackPoint WHERE altitude != -777) AS t

# Second select (#2) calculates the difference in altitude between the current trackpoint and the previous one based on the valid trackpoints
# SELECT activity_id, altitude - LAG(altitude) OVER 
# (PARTITION BY activity_id ORDER BY date_time) AS difference FROM (#1) AS t) 

# Third select (#3) calculates the sum of the difference for each activityif the difference is positiv based on the tip given in the task
# SELECT activity_id, SUM(difference) AS activity_altitude FROM (#2) 
# AS altitude_difference WHERE difference > 0 GROUP BY activity_id

# Fourth select (#4) sum each user's activity differences in altitude and multiplies it with 0.304 to convert it to meters
# SELECT RANK() OVER (
#         ORDER BY SUM(activity_altitude)*0.304 DESC
#         ) AS Top, user_id, SUM(activity_altitude)*0.304 as altitude_in_meters FROM Activity JOIN 
#         (#3) AS difference_table 
#         ON Activity.id = difference_table.activity_id GROUP BY user_id ORDER BY altitude_in_meters DESC 
#         LIMIT 15

query = """SELECT RANK() OVER (
        ORDER BY SUM(activity_altitude)*0.304 DESC
        ) AS Top, user_id, SUM(activity_altitude)*0.304 as altitude_in_meters FROM Activity JOIN 
        (SELECT activity_id, SUM(difference) AS activity_altitude FROM 
        (SELECT activity_id, altitude - LAG(altitude) OVER 
        (PARTITION BY activity_id ORDER BY date_time) AS difference FROM 
        (SELECT activity_id, altitude, date_time FROM TrackPoint WHERE altitude != -777) AS t) 
        AS altitude_difference WHERE difference > 0 GROUP BY activity_id) AS difference_table 
        ON Activity.id = difference_table.activity_id GROUP BY user_id ORDER BY altitude_in_meters DESC 
        LIMIT 15"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Top 15 users who have gained the most altitude meters")
print(tabulate(rows, headers=db.cursor.column_names))

Top 15 users who have gained the most altitude meters
  Top    user_id    altitude_in_meters
-----  ---------  --------------------
    1        004              331165
    2        041              240127
    3        003              233050
    4        030              175219
    5        128              152571
    6        039              146319
    7        000              121186
    8        002              114761
    9        025              108862
   10        037               98960.5
   11        140               94589.9
   12        062               74559
   13        017               62402.1
   14        034               61278.5
   15        042               61159


### Task 10

Find the users that have traveled the longest total distance in one day for each
transportation mode.

The first approach, and the one we want to present, search for the longest distance by a user in one day (we interpreted it as one date day, like in task 7 where it did not go over to the next day). This implementation bases on that if a user has multiple activities of the same transportation mode within a day, then the distance for these activities will be summed up.

In [128]:
# Find the users that have traveld the longest total distance in one day for each transportation mode

from haversine import haversine, Unit
 
query = """
SELECT a.id, tp.lat, tp.lon, CAST(tp.date_time AS date) AS day, a.transportation_mode, a.user_id
FROM Activity a
INNER JOIN TrackPoint tp
ON a.id = tp.activity_id
WHERE a.transportation_mode IS NOT NULL
ORDER BY a.user_id, a.transportation_mode, tp.date_time ASC
"""

db.cursor.execute(query)
result = db.cursor.fetchall()

In [129]:
activities = {} # This contains an overview of all the activities for each user for each date

"""
activities = {
    user_id: {
        transportation_mode: {
            date: {
                distance: int,
                position: (lat, lon),
            }
        }
    }
}
"""

for res in result:
    activity_id = res[0]
    lat = res[1]
    lon = res[2]
    date = res[3]
    transportation_mode = res[4]
    user_id = res[5]

    # Checks if user id is in the activities dictionary, if not, add it
    if user_id not in activities:
        activities[user_id] = {}

    # Checks if transportation mode is in the user dictionary, if not, add it
    if transportation_mode not in activities[user_id]:
        activities[user_id][transportation_mode] = {}

    # Checks if date is in the transportation mode dictionary, if not, add it
    if date not in activities[user_id][transportation_mode]:
        activities[user_id][transportation_mode][date] = {
            "distance": 0,
            "position": (lat, lon),
            "activity_id": activity_id
        }

    # Checks if it is the same position as previously or that it is another activity then skip
    if activities[user_id][transportation_mode][date]["position"] == (lat, lon) or \
    activities[user_id][transportation_mode][date]["activity_id"] != activity_id:
        # Keep the current position to calculate the distance for the next row
        activities[user_id][transportation_mode][date]["position"] = (lat, lon)
        # Keep the current activity_id so that we don't calculate the distance between two different activities
        activities[user_id][transportation_mode][date]["activity_id"] = activity_id   
        continue
    # Calculates the distance between the current position and the previous position
    activities[user_id][transportation_mode][date]["distance"] += haversine(
        activities[user_id][transportation_mode][date]["position"], (lat, lon), unit=Unit.KILOMETERS)

    # Keep the current position to calculate the distance for the next row
    activities[user_id][transportation_mode][date]["position"] = (lat, lon)
    # Keep the current activity_id so that we don't calculate the distance between two different activities
    activities[user_id][transportation_mode][date]["activity_id"] = activity_id    
longest_activities = {} # This contains the longest activity for each transportation mode 

# Loop through the user date activities
for user_id in activities:
    for transportation_mode in activities[user_id]:
        for date in activities[user_id][transportation_mode]:

            # Check if there's no dates of users with that activity type
            # Add the current one as the longest distance activity
            if transportation_mode not in longest_activities:
                longest_activities[transportation_mode] = {
                    "distance": activities[user_id][transportation_mode][date]["distance"],
                    "user_id": user_id,
                }
            # Else check if the current one is longer, if so, then replace   
            elif longest_activities[transportation_mode]["distance"] < activities[user_id][transportation_mode][date]["distance"]:
                longest_activities[transportation_mode] = {
                    "distance": activities[user_id][transportation_mode][date]["distance"],
                    "user_id": user_id,
                }

# Create a table with the longest activity for each transportation mode
# to present the result 
users = []
for transportation_mode in longest_activities:
    distance = longest_activities[transportation_mode]["distance"]
    user_id = longest_activities[transportation_mode]["user_id"]
    users.append((user_id, transportation_mode, distance))

table = [["User", "Transportation mode", "Distance (km)"]]
table.extend(users)

print(tabulate(table, headers="firstrow"))

  User  Transportation mode      Distance (km)
------  ---------------------  ---------------
   128  bus                        207.413
   128  taxi                        40.2233
   128  bike                        63.1124
   108  walk                        22.8074
   128  car                        398.173
   062  run                          0.0332532
   062  train                      277.258
   128  subway                      33.9351
   128  airplane                  2527.12
   128  boat                        65.5548


Another approach we have done before ending up with the previous one is the one when we first interpreted it as that we are tied up to one activity and check if the hours between start and endtime are not over 24 hours. However, afterwards we find this approach quite restricted to one activity and does not really answer the question about travelling the most distance in one day.

Note: When it comes to code, it was written a bit differently (with less for loops), reason being that the "run" transportation mode was suspiciously low, so did not want to use the same approach (or code wise with regards of for loops and dict inside dict inside dict etc.) to ensure that the code was correct (at least one of them).

In [130]:
from haversine import haversine, Unit

query = """
SELECT a.id, tp.lat, tp.lon, a.transportation_mode, a.user_id
FROM Activity a
INNER JOIN TrackPoint tp ON a.id = tp.activity_id 
WHERE (TIMESTAMPDIFF(SECOND, a.start_date_time, a.end_date_time)) < 60 * 60 * 24
AND a.transportation_mode IS NOT NULL
ORDER BY a.id, tp.date_time ASC;
"""

db.cursor.execute(query)
result = db.cursor.fetchall()

# Keeps track of the activities
activity_information = {} # dict: activity_id: [user_id, transportation_mode, distance]
previous_activity_position = {} # Keep track of the last position of each activity

activity_distance = {} # Keep track of the distance of each activity
transportation = {} # Keep track of the transportation mode of each activity
activity_user = {} # Keep track of the user of each activity

# Go through all the activities and calculate the distance between the current and last position
for res in result:
    # Read from query result
    activity_id = res[0]
    current_position = (res[1], res[2])
    transportation_mode = res[3]
    user_id = res[4]

    # Set the previous positions
    previous_position = previous_activity_position.get(activity_id, current_position)
    previous_activity_position[activity_id] = current_position # Update the last position

    # Calculate the distance
    current_activity_information = activity_information.get(activity_id, [0,0,0])
    distance = current_activity_information[2] + haversine(current_position, previous_position, unit=Unit.KILOMETERS)

    # Update the activity information
    activity_information[activity_id] = [user_id, transportation_mode, distance]

# Find the longest activity for each transportation mode
longest_activities = {} # dict: transportation_mode: [distance, user_id, activity_id]
for activity_id, info in activity_information.items():
    user_id, transportation_mode, distance = info
    previous_longest_activity = longest_activities.get(transportation_mode, [0,0,0])

    # Check if activity is longer
    if previous_longest_activity[0] > distance:
        continue

    # Update longest activity
    longest_activities[transportation_mode] = [distance, user_id, activity_id]


# Create a table with the longest activity for each transportation mode
users = []
for transportation_mode, items in longest_activities.items():
    distance, user_id, activity_id = items
    users.append((user_id, transportation_mode, distance))

table = [["User", "Transportation mode", "Distance (km)"]]
table.extend(users)

print(tabulate(table, headers="firstrow"))

  User  Transportation mode      Distance (km)
------  ---------------------  ---------------
   128  bus                        207.413
   128  taxi                        40.2233
   139  walk                        22.8074
   128  bike                        52.5372
   128  car                        331.418
   062  run                          0.0332532
   062  train                      277.258
   128  subway                      23.2978
   128  airplane                  4391.42
   128  boat                        65.5548


The last approach we did is for interpreting one day as 24 hours intervals. The solution below however is quite time consuming, and we did not get it to finish running.

In [ ]:
# Retrieve all the trackpoints and the relevant information needed
query = """
    SELECT a.id, tp.date_time, tp.lat, tp.lon, a.transportation_mode, a.user_id
    FROM Activity a
    INNER JOIN TrackPoint tp ON a.id = tp.activity_id 
    WHERE a.transportation_mode IS NOT NULL
    ORDER BY user_id, a.id ASC, tp.date_time ASC;
"""

db.cursor.execute(query)
result = db.cursor.fetchall()

# Initialize a structure to keep track of the <24h interval
tracker_24h = {}
id_counter = 0
# Loop through all the trackpoints
for i in result:
    dt1 = i[1]
    tracker_24h[id_counter] = {
        "user_id": i[5],
        "transportation_mode": i[4],
        "position": (i[2], i[3]),
        "activity_id": i[0],
        "distance": 0.0
    }
    
    # Loop through all the trackpoints
    for j in result:
        dt2 = j[1]
        delta = dt2-dt1

        # If the date difference between the two trackpoints are more than 24 hours then break the interval
        # And start on new interval
        hours_delta = delta.days*24+delta.seconds/3600
        if hours_delta > 24:
            break

        # If it is from the same activity, add the distance between the two trackpoints to the interval information
        if tracker_24h[id_counter]['activity_id'] == j[0]:
            distance = haversine(tracker_24h[id_counter]['position'], (j[2], j[3]), unit=Unit.KILOMETERS)
            tracker_24h[id_counter]['distance'] += distance
            tracker_24h[id_counter]['position'] = (j[2], j[3])
            tracker_24h[id_counter]['activity_id'] = j[0]
        # If it is not then skip adding distance
        else:
            tracker_24h[id_counter]['position'] = (j[2], j[3])
            tracker_24h[id_counter]['activity_id'] = j[0]
            continue
    id_counter += 1

transportation_modes = {}
# Loop through all the intervals to check for each transportation mode
# Which interval has the longest distance
for _, interval in tracker_24h:
    if interval['transportation_mode'] not in transportation_modes:
        transportation_modes[interval['transportation_mode']] = interval
    else:
        if interval['distance'] > transportation_modes[interval['transportation_mode']]['distance']:
            transportation_modes[interval['transportation_mode']] = interval

# Create a table with the longest activity for each transportation mode
users = []
for transportation_mode, items in transportation_modes.items():
    user_id, transportation_mode, _, distance = items
    users.append((user_id, transportation_mode, distance))

table = [["User", "Transportation mode", "Distance (km)"]]
table.extend(users)

print(tabulate(table, headers="firstrow"))

### Task 11

Find all users who have invalid activities, and the number of invalid activities per user.

In [131]:
# Assuming that the trackpoints are after order in the datetime when inserting
# Thus id is also in order. Then check by using id - 1 to see if the date_time between the trackpoint
# id and id-1 have a interval larger than 5 minutes

query = """SELECT a.user_id, count(*)
                FROM Activity a
                JOIN TrackPoint t1
                ON a.id = t1.activity_id
                JOIN TrackPoint t2
                ON t1.id = t2.id-1
                AND t1.activity_id = t2.activity_id
                WHERE t2.date_time > t1.date_time + INTERVAL 5 MINUTE
                AND t1.id != t2.id
                GROUP BY a.user_id
                ORDER BY a.user_id ASC"""

db.cursor.execute(query)
rows = db.cursor.fetchall()
print(tabulate(rows, headers=db.cursor.column_names))

  user_id    count(*)
---------  ----------
      000         442
      001         116
      002         295
      003         839
      004        1169
      005         126
      006          48
      007         133
      008          33
      009         114
      011          38
      012         163
      013          68
      014         328
      015         136
      016          60
      017         492
      018          97
      019          65
      020          15
      021           1
      022         188
      023          30
      024          73
      025         424
      026          51
      027          13
      028          90
      029          37
      030         509
      031           7
      032          36
      033           3
      034         240
      035         122
      036         125
      037         279
      038         264
      039         423
      040          50
      041         386
      042         117
      043          72
      044 

### Task 12

Find all the users who have registered transportation_mode and their most used transportation_mode

Comment on implementation: Although with this dataset, does not need to join User, however in cases where user does not have labels, but still has some activities in Activity with transportation_mode then a join is necessary. For users that have the same number of activities tagged on multiple transportation mode, we've decided to take the first transportation mode in alphabetical order.

In [136]:
# First select (#1) finds the order of the most used transportation modes for each user
# if there exists more than one transportation mode for the largest value, then takes the one 
# with the alphabetical order. Defines a row number with the ordering of the most used transportation mode
# for later use. This will not count the ones with mode equals NULL:
# SELECT user_id, transportation_mode, ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY COUNT(*) DESC, transportation_mode ASC) 
#         AS rownum FROM Activity JOIN User ON Activity.user_id = User.id WHERE transportation_mode IS NOT NULL 
#         AND User.has_labels=TRUE GROUP BY user_id, transportation_mode

# Second select (#2) uses the row number from #1 to retrieve the most used transportation mode
# SELECT user_id, transportation_mode FROM (#1)
#         AS activity_grouped WHERE rownum=1 ORDER BY user_id

query = """SELECT user_id, transportation_mode as most_used_transportation_mode FROM 
        (SELECT user_id, transportation_mode, ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY COUNT(*) DESC, transportation_mode ASC) 
        AS rownum FROM Activity JOIN User ON Activity.user_id = User.id WHERE transportation_mode IS NOT NULL 
        AND User.has_labels=TRUE GROUP BY user_id, transportation_mode)
        AS activity_grouped WHERE rownum=1 ORDER BY user_id"""
db.cursor.execute(query)
rows = db.cursor.fetchall()
print("Find all the users who have registered transportation_mode and their most used transportation_mode")
print(tabulate(rows, headers=db.cursor.column_names))

Find all the users who have registered transportation_mode and their most used transportation_mode
  user_id  most_used_transportation_mode
---------  -------------------------------
      010  taxi
      020  bike
      021  walk
      052  bus
      056  bike
      058  car
      060  walk
      062  bus
      064  bike
      065  bike
      067  walk
      069  bike
      073  walk
      075  walk
      076  car
      078  walk
      080  bike
      081  bike
      082  walk
      084  walk
      085  walk
      086  car
      087  walk
      089  car
      091  bus
      092  bus
      097  bike
      098  taxi
      101  car
      102  bike
      107  walk
      108  walk
      111  taxi
      112  walk
      115  car
      117  walk
      125  bike
      126  bike
      128  car
      136  walk
      138  bike
      139  bike
      144  walk
      153  walk
      161  walk
      163  bike
      167  bike
      175  bus


### End

In [133]:
# Closing the connection
db.close_connection()


-----------------------------------------------
Connection to 8.0.34-0ubuntu0.22.04.1 is closed
